# 암호화 대비 SQL 데이터 출력 클래스 제작

## 구현 방식
SQL 접속용 클래스 PreprocessQuery를 상속한 SelectData, JoinData 클래스 구현

In [2]:
import pymysql
from sqlalchemy import create_engine
from abc import ABC, abstractmethod
from typing import *
import re

from prettytable import PrettyTable
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

### 기존 PreprocessQuery 클래스

In [2]:
class PreprocessQuery:
    """데이터 전처리 목적의 SQL 쿼리 클래스"""
    def __init__(self, pw):
        self._pw = pw
        self.connection = None
        self.Cursor = None
        self.SQL = None
    
    def connectMySQL(self, 
                     serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        """MySQL DBMS 데이터베이스에 접속: 서버IP주소, 사용자명, 계정 암호, 데이터베이스명, 한글 인코딩 방식"""
        try:
            self.connection = pymysql.connect(
                host = serverIP, port = port_num,
                user = user_name, password = self._pw,
                db = database_name, charset = kr_encoder
            )
            self.Cursor = self.makeCursor(self.connection)
        except pymysql.Error as e:
            print(f"Error Connecting to MySQL from Python: {e}")
    
    def makeCursor(self, connect):
        """커서 생성"""
        return connect.cursor()
    
    def dataQueryLanguage(self, sql):
        """SQL 쿼리문 작성"""
        self.SQL = f"{sql}"
    
    def queryExecute(self):
        """SQL 쿼리문 실행 및 예외처리"""
        try:
            self.Cursor.execute(self.SQL)
            actionOutput = self.Cursor.fetchall()
            return actionOutput
        except pymysql.Error as e:
            print(f"Error Executing Query: {e}")
    
    def queryCommit(self):
        """실행 결과 확정"""
        self.Cursor.execute(self.SQL)
        self.Cursor.commit()
    
    def closeConnection(self):
        """연결 및 커서 닫기"""
        if self.Cursor:
            self.Cursor.close()
        if self.connection:
            self.connection.close()

### SelectData 클래스

In [3]:
class SelectData(PreprocessQuery):
    def __init__(self, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str, pw: str):
        self.connection = None
        self.Cursor = None
        self.SQL = None
        self._serverIP = serverIP
        self._port_num = port_num
        self._user_name = user_name
        self._database_name = database_name
        self._kr_encoder = kr_encoder
        self._pw = pw
    
    def selectTable(self, table: str):
        self.connectMySQL(self._serverIP, self._port_num, self._user_name, self._database_name, self._kr_encoder)
        self.dataQueryLanguage(f"SELECT * FROM {table}")
        results = self.queryExecute()
        return results
    
    def printTable(self, data):
        """테이블 출력 결과에 PrettyTable 적용하는 함수"""
        if data:
            columns = [ [desc[0] for desc in self.Cursor.description] ]
            table = PrettyTable(*columns)
    
            for row in data:
                row_list = list(row)
                table.add_row(row_list)
            return table
        return False
    
    def concatColumns(self, columns: tuple, table: str):
        """SELECT CONCAT(column1, column2, column3) FROM TABLE 구현 함수"""
        pass

In [6]:
d = SelectData(serverIP = "localhost", port_num = 3306, user_name = "root", database_name = "FINANCIALCONSUMER", kr_encoder = "utf8", pw ="1234")
ds = d.selectTable(table = "DATA_JOIN_ACCOMODATIONAPP")
print(ds[0:10])

((1, '고대옥', datetime.date(1966, 4, 1), 'female', '고대옥@gmail.com', '010-5978-9544', 'N', '전라북도 익산시 서동로 464(용제동)', datetime.date(2021, 7, 20)), (2, '사미소', datetime.date(1991, 1, 12), 'female', '사미소@outlook.com', '010-3920-5092', 'N', '제주특별자치도 제주시 조천읍 신촌남8길 87', datetime.date(2020, 6, 6)), (3, '서회걸', datetime.date(1950, 9, 20), 'male', '서회걸@naver.com', '010-7209-1680', 'Y', '경상북도 영천시 신녕면 찰방길 28', datetime.date(2022, 2, 17)), (4, '장예홍', datetime.date(2002, 11, 22), 'male', '장예홍@naver.com', '010-3093-9161', 'Y', '경기도 성남시 수정구 시민로 172(신흥동)', datetime.date(2021, 11, 4)), (5, '표재창', datetime.date(1983, 8, 21), 'male', '표재창@nate.com', '010-2443-6530', 'Y', '경상남도 의령군 지정면 기강로2길 5', datetime.date(2020, 8, 20)), (6, '주동지', datetime.date(1981, 9, 11), 'female', '주동지@naver.com', '010-4437-3442', 'Y', '경기도 이천시 장호원읍 서동대로8369번길 166-13', datetime.date(2020, 11, 13)), (7, '방선두', datetime.date(1954, 3, 22), 'male', '방선두@naver.com', '010-8876-2788', 'Y', '강원특별자치도 태백시 피내골길 11(철암동)', datetime.date(2022, 3, 20)

### concatColumns 구현 방안 : SQLAlchemy ORM Session 활용
- 목적 : ```SELECT CONCAT(NUM_SERIAL, NAME, BIRTH_DATE) FROM DATA_JOIN_ACCOMODATIONAPP``` 과 같은 형태의 구문 구현
- 현재 DataQueryLanguage 활용으로는 ```SELECT CONCAT('NUM_SERIAL', 'NAME', 'BIRTH_DATE') FROM DATA_JOIN_ACCOMODATIONAPP``` 형태가 구현됨.
- 대안으로 SQLAlchemy ORM Session 기능을 활용하여 컬럼 단위로 데이터를 선택하는 코드 제안
- 해당 Session 기능 활용 위해서는 PreprocessQuery의 connection 기능을 SQLAlchemy 기반으로 수정 보완해야 함
    - 참고 : https://soogoonsoogoonpythonists.github.io/sqlalchemy-for-pythonist/tutorial/5.1.%20Core%EC%99%80%20ORM%20%EB%B0%A9%EC%8B%9D%EC%9C%BC%EB%A1%9C%20%ED%96%89%20%EC%A1%B0%ED%9A%8C%ED%95%98%EA%B8%B0.html#select-%E1%84%85%E1%85%B3%E1%86%AF-%E1%84%90%E1%85%A9%E1%86%BC%E1%84%92%E1%85%A1%E1%86%AB-sql-%E1%84%91%E1%85%AD%E1%84%92%E1%85%A7%E1%86%AB%E1%84%89%E1%85%B5%E1%86%A8-%E1%84%80%E1%85%AE%E1%84%89%E1%85%A5%E1%86%BC

In [3]:
from sqlalchemy import create_engine
import pymysql
from sqlalchemy.orm import Session
from urllib.parse import quote
from sqlalchemy import select, func
from sqlalchemy.dialects import mysql

In [4]:
user = "root"
pwd = "1234"
host = "localhost"
port = 3306
db = "FINANCIALCONSUMER"

db_url = f'mysql+pymysql://{user}:{quote(pwd)}@{host}:{port}/{db}'
engine = create_engine(db_url, echo=True)

session 활용 리턴값 row의 경우, tuple과 데이터 형태가 유사하지만 ```row.NUM_SERIAL``` 과 같은 형식으로 데이터를 컬럼별로 뽑아낼 수 있음
    - 기존 PreprocessQuery는 데이터 리턴값이 tuple 타입으로, 데이터 컬럼별 분류가 어려움

In [5]:
with Session(engine) as session:
    for row in session.execute("SELECT * FROM DATA_JOIN_CARDPAYMENT limit 0, 10"):
        print(str(row.NUM_SERIAL) + str(row.NAME) + str(row.BIRTH_DATE))

2024-02-02 10:14:17,008 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-02-02 10:14:17,008 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-02 10:14:17,011 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-02-02 10:14:17,011 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-02 10:14:17,013 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-02-02 10:14:17,013 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-02 10:14:17,014 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-02 10:14:17,015 INFO sqlalchemy.engine.Engine SELECT * FROM DATA_JOIN_CARDPAYMENT limit 0, 10
2024-02-02 10:14:17,015 INFO sqlalchemy.engine.Engine [generated in 0.00042s] {}
1국회옥1996-08-30
2윤창성1993-08-27
3신류안1968-02-28
4류백겸2000-06-17
6강이비1952-08-17
7남복조1960-12-25
10탁한음1950-06-05
11윤태흠1993-12-01
12지채문1960-03-30
13레건영1991-07-10
2024-02-02 10:14:17,016 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
results = []

with Session(engine) as session:
    for row in session.execute("SELECT * FROM DATA_JOIN_CARDPAYMENT limit 0, 10"):
        results.append(str(row.NUM_SERIAL) + str(row.NAME) + str(row.BIRTH_DATE))

2024-02-02 10:14:18,635 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-02 10:14:18,636 INFO sqlalchemy.engine.Engine SELECT * FROM DATA_JOIN_CARDPAYMENT limit 0, 10
2024-02-02 10:14:18,637 INFO sqlalchemy.engine.Engine [cached since 1.622s ago] {}
2024-02-02 10:14:18,642 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
results

['1국회옥1996-08-30',
 '2윤창성1993-08-27',
 '3신류안1968-02-28',
 '4류백겸2000-06-17',
 '6강이비1952-08-17',
 '7남복조1960-12-25',
 '10탁한음1950-06-05',
 '11윤태흠1993-12-01',
 '12지채문1960-03-30',
 '13레건영1991-07-10']

In [8]:
results[0]  # 해당 형태의 데이터를 기반으로 암호화 클래스 제작 돌입

'1국회옥1996-08-30'

# 암호화 클래스 제작

In [7]:
import os
from hashlib import blake2b
from hashlib import sha256

In [14]:
class EncryptData:
    """데이터 전처리 및 합치기 클래스에서 출력된 데이터를 해싱하는 클래스"""
    def __init__(self, data):
        self._data = data
        
    def addSalt(self, data):
        """BLAKE2B 알고리즘으로 데이터에 Salt값을 붙이는 함수"""
        results = []
        
        for i in range(len(data)):
            d = data[i].encode('utf-8')
            salt = os.urandom(blake2b.SALT_SIZE)
            h = blake2b(salt = salt)
            updateData = h.update(d)
            results.append(updateData)
            
        return results
        
    def makeHash(self, data):
        """데이터 합치기, Salt값 더하기 작업 완료된 데이터를 SHA256 알고리즘으로 해싱하는 함수"""
        results = []
        
        for i in range(len(data)):
            d = data[i].encode('utf-8')
            result = sha256(d)
            results.append(result)
            
        return results
    
    def printResult(self, data):
        """해싱 완료된 민감정보 데이터를 같은 인덱스의 다른 데이터와 합쳐서 프린트하는 함수"""
        pass

In [9]:
d2 = "1, '고대옥', datetime.date(1966, 4, 1), 'female', '고대옥@gmail.com', '010-5978-9544', 'N', '전라북도 익산시 서동로 464(용제동)', datetime.date(2021, 7, 20)"

In [13]:
d3 = EncryptData(d2)